In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from preprocessing import clsf_boz_cd, clsf_rrnr_dvcd, clsf_dmfr_dvcd, clsf_crd_grd

In [2]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format
os.makedirs('result', exist_ok=True)

# 전역변수
FILE_PATH = Path('./data/현행추정부채_일반')

In [6]:
# 데이터 불러오기
일반_상품정보 = pd.read_excel(FILE_PATH / '일반_상품정보.xlsx', dtype={'PDC_CD': str, 'PDGR_CD': str})
일반_원수_미경과보험료 = pd.read_excel(FILE_PATH / '일반_원수_미경과보험료.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str})
일반_출재_미경과보험료 = pd.read_excel(FILE_PATH / '일반_출재_미경과보험료.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str, 'T02_RN_RINSC_CD': str})
재보험자_국내신용등급 = pd.read_excel(FILE_PATH / '재보험자_국내신용등급.xlsx', dtype={'재보험사코드': str}) \
    .rename(columns = {'재보험사코드': 'T02_RN_RINSC_CD', '국내신용등급': 'CRD_GRD'})

In [7]:
# 데이터 전처리
일반_원수_미경과보험료['BOZ_CD'] = clsf_boz_cd(일반_원수_미경과보험료, 일반_상품정보)
일반_원수_미경과보험료['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_미경과보험료, '원수')
일반_원수_미경과보험료['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_미경과보험료)

일반_출재_미경과보험료['BOZ_CD'] = clsf_boz_cd(일반_출재_미경과보험료, 일반_상품정보)
일반_출재_미경과보험료['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_출재_미경과보험료, '출재')
일반_출재_미경과보험료['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_미경과보험료)
일반_출재_미경과보험료['CRD_GRD'] = clsf_crd_grd(일반_출재_미경과보험료, 재보험자_국내신용등급)
일반_출재_미경과보험료

,RRNR_DAT_DVCD,RRNR_CTC_BZ_DVCD,NTNL_CTRY_CD,ARC_INPL_CD,T02_RN_RINSC_CD,T02_LTPD_RN_URND_PRM,BOZ_CD,RRNR_DVCD,DMFR_DVCD,CRD_GRD
0,01,3,KR,1040110,124012,244906286,A001,03,01,AA
1,01,3,KR,1054410,124012,277457,A007,03,01,AA
2,01,3,KR,1049210,124060,3997104940,A006,03,01,AAA
3,01,3,KR,1067110,124012,13935049,A007,03,01,AA
4,01,3,KR,1061610,122086,35489513,A007,03,01,AA-
...,...,...,...,...,...,...,...,...,...,...
1713,01,3,KR,1075010,511229,159905282,A004,03,01,AA+
1714,01,3,KR,1050310,511098,8226012,A003,03,01,AA
1715,01,3,KR,1050310,511127,5484008,A003,03,01,AA
1716,01,3,KR,1050310,511279,2742004,A003,03,01,AA


In [16]:
# 데이터 집계
일반_원수_미경과보험료_집계 = 일반_원수_미경과보험료.groupby(['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD'])['LTPD_URND_PRM'].sum().reset_index()
일반_원수_미경과보험료_집계

,RRNR_DVCD,DMFR_DVCD,BOZ_CD,LTPD_URND_PRM
0,01,01,A001,23420916171
1,01,01,A002,27146263273
2,01,01,A003,96625029260
3,01,01,A004,22191701960
4,01,01,A005,5095840261
5,01,01,A006,66353121310
6,01,01,A007,93981583216
7,01,01,A008,4773218318
8,01,01,A010,73720023337
9,01,01,A011,3516025


In [ ]:
# 데이터 내보내기